In [1]:
import pandas as pd
import numpy as np

In [3]:
data = {"x":np.random.randn(10), "y":np.random.randn(10), "values":[1,1,2,2,3,3,4,4,5,5]}

In [4]:
df = pd.DataFrame(data)

In [5]:
df

,x,y,values
0,0.086407,1.035410,1
1,-1.414773,1.415566,1
2,0.750165,0.143831,2
3,0.126430,0.015483,2
4,-0.312779,1.626497,3
5,-1.060493,0.056935,3
6,0.636196,1.345945,4
7,-1.383807,-0.401065,4
8,0.483825,-0.686364,5
9,0.327340,1.147816,5


In [46]:
xs = df['x'].values[0:4]
ys = df['y'].values[0:4]
coords = np.stack([xs, ys])

In [52]:
coords.shape

(2, 4)

In [59]:
xs

array([ 0.08640731, -1.41477307,  0.75016536,  0.12643045])

In [71]:

order = np.argsort(xs)
print(order)
tmp = order[-1]
order[-1] = order[-2]
order[-2] = tmp
print(order)

[1 0 3 2]
[1 0 2 3]


In [6]:
df['values'].values

array([1, 1, 2, 2, 3, 3, 4, 4, 5, 5])

In [8]:
df.loc[df['values']==1]

,x,y,values
0,0.086407,1.035410,1
1,-1.414773,1.415566,1


In [7]:
np.unique(df['values'].values)

array([1, 2, 3, 4, 5])

In [14]:
for value_id in np.unique(df['values'].values):
    pixels = df.loc[df['values'] == value_id]
    xs = pixels['x'].values
    ys = pixels['y'].values
    coords = [[xi, yi] for (xi, yi) in zip(xs, ys)]
    
    features = {
      "type": "Feature",  
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          coords
        ]
      },
      "properties": {
        "id": "roi_1"  
      }
    }
    print(features)
    
    

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[0.08640730668109275, 1.0354102448979745], [-1.4147730743792366, 1.4155656740492766]]]}, 'properties': {'id': 'roi_1'}}
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[0.7501653643926642, 0.1438314685158929], [0.1264304541668935, 0.015482766652620397]]]}, 'properties': {'id': 'roi_1'}}
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-0.31277900757659804, 1.6264974523688789], [-1.060492987374343, 0.056935060327278866]]]}, 'properties': {'id': 'roi_1'}}
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[0.6361960629073234, 1.345944601874529], [-1.383806506742293, -0.40106477961991305]]]}, 'properties': {'id': 'roi_1'}}
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[0.483825469844522, -0.6863640557458588], [0.3273399618733337, 1.147815504835722]]]}, 'properties': {'id': 'roi_1'}}


In [26]:
! pip install geojson

In [88]:
import pandas as pd
import json
from geojson import GeoJSON


def convert_csv_to_geojson(input_file, output_file):

    # Read the CSV data into a pandas DataFrame

    cols = ['x', 'y', 'values'] 
    df = pd.read_csv(input_file, names=cols)
    
    print(df.keys())

    # Create a GeoJSON object

    geojson = {

        'type': 'FeatureCollection',

        'features': []

    }

    # Loop through each row in the DataFrame and convert to a GeoJSON feature

    for value_id in np.unique(df['values'].values):

        pixels = df.loc[df['values'] == value_id]

        xs = pixels['x'].values

        ys = pixels['y'].values
        
        order = np.argsort(xs)
        tmp = order[-1]
        order[-1] = order[-2]
        order[-2] = tmp
        # print(order)
        
        xs = xs[order]
        ys = ys[order]

        coords = [[int(xi), int(yi)] for (xi, yi) in zip(np.append(xs, np.array(xs[0])), np.append(ys, np.array(ys[0])))] 

        properties = {

        'name': str(value_id),

        'description': "a pixel"

        }

        features = {

        "type": "Feature",  
        "id": f"roi_{value_id}",
        "originalObjectId": str(value_id),
        "otherProperty": str(value_id),
        "name": str(value_id),

        "geometry": {

        "type": "Polygon",

        "coordinates": [

        coords

        ]

        },

        "properties": {
            "objectType": "annotation",
            "originalObjectId": str(value_id),
            "otherProperty": str(value_id),
            "name": str(value_id),
        }

            }

        #print(features)

        geojson['features'].append(features)
    
    with open(output_file, "w") as geojson_file:
        json.dump(geojson, geojson_file, indent=2)
    

    # with open(output_file, 'w') as outfile:
    #     GeoJSON.dump(geojson,output_file)


# Example usage

input_file = '~/Downloads/new_csv_file.csv'

output_file = 'output.geojson'

convert_csv_to_geojson(input_file, output_file)


Index(['x', 'y', 'values'], dtype='object')
